# Margins for NSE
-[ ] Build / Extract the chains    
-[ ] Get undPrices   
-[ ] Integrate undPrices to the chains   

-[ ] Get the lots   
-[ ] Get margins for with qty as the lot   
-[ ] Make logic for margins from NSE rules   
-[ ] For margins with NaN, replace margins with the one derived from logic   
-[ ] Integrate margins to chains

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [2]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# Suppress Errors
util.logToFile(DATAPATH.parent.parent / 'log' / 'ztest.log')

## Get underlying contracts and prices

In [3]:
# imports
import asyncio
from utils import get_pickle, get_mkt_prices, pickle_me, get_file_age

In [4]:
# Get unds
unds = get_pickle(DATAPATH / 'unds.pkl')
df_unds = pd.DataFrame.from_dict(unds.items())
df_unds.columns = ['symbol', 'contract']

# Get und prices
# ... check file age.
UND_PRICE_PATH = DATAPATH / 'df_und_prices.pkl'
und_price_file_age = get_file_age(UND_PRICE_PATH)
if not und_price_file_age:
    age_in_mins = 30
else:
    age_in_mins = und_price_file_age.td.total_seconds()/60

if age_in_mins >= 30:
    df_und_prices = asyncio.run(get_mkt_prices(port=PORT, contracts=df_unds.contract))
    pickle_me(df_und_prices, UND_PRICE_PATH)
else:
    df_und_prices = get_pickle(UND_PRICE_PATH)

Getting market prices with IVs: 100%|██████████| 5/5 [01:32<00:00, 18.40s/it]


## Build / Extract the chains

In [5]:
# imports
import asyncio
from utils import make_chains, pickle_me

# inputs
contracts = df_unds.contract

In [6]:
# Get chains
CHAIN_PATH = DATAPATH / 'df_chains.pkl'
chains_file_age = get_file_age(CHAIN_PATH)

if not chains_file_age:
    age_in_days = 1
else:
    age_in_days = chains_file_age.td.total_seconds()/60/60/12
    
if age_in_days >= 1:
    df_chains = asyncio.run(make_chains(port=PORT, contracts=contracts, MARKET=MARKET))
    pickle_me(df_chains, CHAIN_PATH)
else:
    df_chains = get_pickle(CHAIN_PATH)

Getting chains: 100%|██████████| 8/8 [01:57<00:00, 14.70s/it]


In [7]:
df_chains

,symbol,undId,expiry,dte,strike,multiplier,localSymbol,undPrice,iv,sigma,strike_sdev,right,exchange
0,NIFTY50,51497778,20240314,2.402567,9000.0,1,NIFTY50,22332.65,0.121449,220.052189,-60.588581,P,NSE
1,NIFTY50,51497778,20240314,2.402567,10000.0,1,NIFTY50,22332.65,0.121449,220.052189,-56.044205,P,NSE
2,NIFTY50,51497778,20240314,2.402567,11000.0,1,NIFTY50,22332.65,0.121449,220.052189,-51.499828,P,NSE
3,NIFTY50,51497778,20240314,2.402567,12000.0,1,NIFTY50,22332.65,0.121449,220.052189,-46.955452,P,NSE
4,NIFTY50,51497778,20240314,2.402567,13000.0,1,NIFTY50,22332.65,0.121449,220.052189,-42.411075,P,NSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63112,BANKBAROD,180222138,20240530,79.401317,370.0,1,BANKBARODA,277.40,0.347298,44.934115,2.060795,C,NSE
63113,BANKBAROD,180222138,20240530,79.401317,375.0,1,BANKBARODA,277.40,0.347298,44.934115,2.172069,C,NSE
63114,BANKBAROD,180222138,20240530,79.401317,380.0,1,BANKBARODA,277.40,0.347298,44.934115,2.283343,C,NSE
63115,BANKBAROD,180222138,20240530,79.401317,385.0,1,BANKBARODA,277.40,0.347298,44.934115,2.394617,C,NSE


## Integrate undPrices and undIV to the chains

In [8]:
und_price_dict = df_und_prices.set_index('symbol').price.dropna().to_dict()
und_iv_dict = df_und_prices.set_index('symbol').iv.dropna().to_dict()

# Replace undPrice and ivs where available
df_chains.undPrice = df_chains.symbol.map(und_price_dict).fillna(df_chains.undPrice)
df_chains.iv = df_chains.symbol.map(und_iv_dict).fillna(df_chains.iv)

## Re-calculate `sigma` and `strike_sdev`

In [9]:
from utils import compute_strike_sd_right

df_chains = compute_strike_sd_right(df_chains)

## Get chains with closest strike price to underlying

In [10]:
# imports
from utils import get_closest_values

In [11]:
def get_strike_closest_to_und(df_chains: pd.DataFrame, 
                              how_many: int= -1) -> pd.DataFrame:
    
    """
    Gets option contracts closest to strike for every expiry\n
    For SNP only the lowest dte is taken\n
    Useful to get reference margins.
    int: -1 for closest Put
    """

    if set(df_chains.exchange.to_numpy()).pop() == 'SNP':
        df_chains = df_chains.loc[df_chains.groupby(['symbol', 'right', 'strike']).dte.idxmin()]\
                            .reset_index(drop=True)

    strk_near_und = df_chains.groupby(['symbol', 'dte'])[['strike', 'undPrice']]\
        .apply(lambda x: get_closest_values(x.strike, 
                                            x.undPrice.min(), 
                                            how_many))
    strk_near_und.name = 'strk_near_und'

    df_ch1 = df_chains.set_index(['symbol', 'dte']).join(strk_near_und)
    df_ch = df_ch1[df_ch1.apply(lambda x: x.strike in x.strk_near_und, axis=1)] \
                            .reset_index()
    
    return df_ch

In [12]:
df_ch = get_strike_closest_to_und(df_chains)

## Get the lots

In [13]:
from utils import get_lots

lots_dict = dict()

for k, v in unds.items():
    if v.exchange.upper() == 'NSE':
        lots_dict[k] = get_lots(v)
    else:
        lots_dict[k] = 1
        
# integrate with df_ch
df_ch['lot'] = df_ch.symbol.map(lots_dict)

## Get margins for with qty as the lot

In [14]:
# imports
import asyncio

from ib_insync import MarketOrder

from utils import get_margins, make_a_raw_contract

opt_contracts = [make_a_raw_contract(symbol=symbol, MARKET=MARKET, secType='OPT', strike=strike, right=right, expiry=expiry)
 for symbol, strike, right, expiry in zip(df_ch.symbol, df_ch.strike, df_ch.right, df_ch.expiry)]

orders = [MarketOrder(action='SELL', totalQuantity=qty) for qty in df_ch.lot]

In [15]:
df_margins = asyncio.run(get_margins(port=PORT, contracts=opt_contracts, orders=orders))

Getting margins:100%|█████████████████████████| 574/574 [02:08<00:00,  4.47it/s]


## Integrate with maximum margin, wherever possible

In [16]:
import numpy as np

In [17]:
# Integration of margins and commissions

cols = ['symbol', 'expiry', 'strike', 'right']
mgn_cols = cols + ['margin', 'comm']
df_chm = pd.merge(df_ch, df_margins[mgn_cols], on=cols, suffixes=[False, '_y']).\
           drop(columns=['strk_near_und'], errors='ignore')

chain_cols = ['symbol', 'expiry']
ch_cols = chain_cols + ['lot', 'margin', 'comm']

df_out = pd.merge(df_chains, df_chm[ch_cols], on=chain_cols, suffixes=[False, '_y'])

# fill missing commissions with max per symbol
commissions = df_chm.groupby('symbol').comm.max().to_dict()
df_out.comm = df_out.comm.fillna(df_out.symbol.map(commissions))

# fill remaining commissions
df_out.comm = df_out.comm.fillna(max(commissions.values()))

# fill margins
mgn_dict = df_out.groupby('symbol').margin.max().to_dict()
cond = df_out.margin.isnull()
df_out.loc[cond, 'margin'] = df_out[cond].symbol.map(mgn_dict)

# make zero margin as nan
zero_margin_condition = df_out.margin == 0
df_out.loc[zero_margin_condition, 'margin'] = np.nan

## Make logic for margins from NSE rules

In [ ]:
# list of options without margins
df1 = df_chm[df_chm.margin.isnull()]
df1.head()


In [ ]:
# Get missing margin percentages from zerodha margin calculator

from io import StringIO
from requests import Session
import re

from utils import get_nse_native_fno_list
from icecream import ic

In [ ]:


# get nse native list
nselist = get_nse_native_fno_list()

In [ ]:
# !!! NO NEED FOR THIS ALREADY THERE IN `localSymbol` field!!!
# create nse symbols for missing margin list

ib2nse = dict()

for ibs in df1.symbol.unique():
    for s in nselist:        
        if ibs in s:
            nsesym = nselist[nselist.index(s)]
            ib2nse[ibs] = s
            
df2 = df1.assign(nse_symbol=df1.symbol.map(ib2nse))

In [ ]:
df2.assign(nse_expiry =df2.nse_symbol + pd.to_datetime(df2.expiry, yearfirst=True).dt.strftime('%d%b').str.upper())

In [ ]:
def zerodah_margin(scrip: str,
                   strike_price: int, 
                   qty: int, 
                   option_type: str = 'PE', 
                   trade: str='sell') -> dict:
    
    """Gets zerodah margins for NSE
    scrip: symbol should be as per NSE eg. `ZEEL24MAR`
    """

    BASE_URL = 'https://zerodha.com/margin-calculator/SPAN'

    payload = {'action': 'calculate',
                'exchange[]': 'NFO',
                'product[]': 'FUT',
                'scrip[]': scrip,   # in `ZEEL24MAR` format
                'option_type[]': 'PE',
                'strike_price[]':280, 
                'qty[]': 2925,
                'trade[]': 'sell'   # 'buy' or 'sell'
    }

    session = Session()

    res = session.post(BASE_URL, data=payload)

    data = res.json()

    return data

## Replace `nan` margins with the one derived from logic